In [5]:
from google.colab import files
import pandas as pd
import io
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import LukeTokenizer, LukeForEntitySpanClassification
import timeit

uploaded = files.upload()


Saving processed_df.csv to processed_df.csv


In [4]:
!pip install flair

     |████████████████████████████████| 319 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 49.9 MB/s 
     |████████████████████████████████| 788 kB 37.6 MB/s 
     |████████████████████████████████| 2.9 MB 34.8 MB/s 
     |████████████████████████████████| 1.2 MB 56.2 MB/s 
     |████████████████████████████████| 48 kB 5.2 MB/s 
     |████████████████████████████████| 19.7 MB 1.1 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 62 kB 819 kB/s 
     |████████████████████████████████| 636 kB 56.3 MB/s 
     |████████████████████████████████| 3.3 MB 52.6 MB/s 
     |████████████████████████████████| 895 kB 28.2 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=72b5b2f07aebb685d96008e3b399d277963c9fe21ab4f6

In [8]:
df = pd.read_csv(io.BytesIO(uploaded['processed_df.csv']), index_col=0)

In [9]:
# load tagger
tagger = SequenceTagger.load("flair/ner-english-large")

def get_flair_result(input_string):
  # make example sentence
  sentence = Sentence(input_string)

  # predict NER tags
  tagger.predict(sentence)
  total_string = ""
  for entity in sentence.get_spans('ner'):
      total_string += str(entity)
      total_string += "\n"
  return total_string


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2021-10-12 12:22:26,123 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [10]:
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/877 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def get_num_entities(input_text):
  split_text = input_text.split("\n")
  n_entities = 0
  for i in split_text:
    split_i = i.split(" ")
    n_entities += (len(split_i) - 1)
  return n_entities

def get_true_num_entities(input_text):
  return len((input_text).split(" "))

def all_person_entities(input_text):
  split_text = input_text.split("\n")
  for i in split_text:
    if len(i) > 0 and i[-3:] != "PER":
      return False

  return True

def get_luke_entity(input_text):
  split_text = input_text.split(" ")
  word_start_positions = [0]
  word_end_positions = [len(split_text[0])]
  for i in split_text[1:]:
    word_start_positions.append(word_end_positions[-1] + 1)
    word_end_positions.append(len(i) + word_start_positions[-1])

  entity_spans = []
  for i, start_pos in enumerate(word_start_positions):
    for end_pos in word_end_positions[i:]:
      entity_spans.append((start_pos, end_pos))

  inputs = tokenizer(input_text, entity_spans=entity_spans, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  predicted_class_indices = logits.argmax(-1).squeeze().tolist()
  if type(predicted_class_indices) == int:
    predicted_class_indices = [predicted_class_indices]
  total_string = ""
  for span, predicted_class_idx in zip(entity_spans, predicted_class_indices):
    if predicted_class_idx != 0:
      total_string += (input_text[span[0]:span[1]] + " " + str(model.config.id2label[predicted_class_idx]) + "\n")
  return total_string

In [12]:
start = timeit.default_timer()
df["luke"] = df["Name"].apply(lambda x: get_luke_entity(x))
stop = timeit.default_timer()
print('LUKE Runtime: {} seconds'.format(stop - start))

start = timeit.default_timer()
df["flair_results"] = df["Name"].apply(lambda x: get_flair_result(x))
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

df["num_luke_entities"] = df["luke"].apply(lambda x: get_num_entities(x))
df["true_num_entities"] = df["Name"].apply(lambda x: get_true_num_entities(x))
df["all_person_entities"] = df["luke"].apply(lambda x: all_person_entities(x))

LUKE Runtime: 11500.698344213 seconds
Flair Runtime: 6788.014681029999 seconds


In [13]:
example_sentences = ["I spoke with {}", "{} went to the store"]

In [18]:
# df.to_csv("df_with_ner.csv")
# files.download('df_with_ner.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# df.loc[(df["all_person_entities"]!= True) | (df["true_num_entities"] != df["num_luke_entities"])]["Race"].value_counts()

In [16]:
# !pip install datasets
# from datasets import load_dataset
# dataset = load_dataset(
#    'conll2003')



# Scratch Code: 

In [17]:
# # load tagger
# tagger = SequenceTagger.load("flair/ner-english")

# # make example sentence
# sentence = Sentence("George Washington went to Washington")

# # predict NER tags
# tagger.predict(sentence)

# # print sentence
# print(sentence)

# # print predicted NER spans
# print('The following NER tags are found:')
# # iterate over entities and print
# for entity in sentence.get_spans('ner'):
#     print(entity)


In [ ]:

# text = "Beyoncé lives in Los Angeles"

# # List all possible entity spans in the text
# word_start_positions = [0, 8, 14, 17, 21]  # character-based start positions of word tokens
# word_end_positions = [7, 13, 16, 20, 28]  # character-based end positions of word tokens
# entity_spans = []
# for i, start_pos in enumerate(word_start_positions):
#   for end_pos in word_end_positions[i:]:
#     entity_spans.append((start_pos, end_pos))

# inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits
# predicted_class_indices = logits.argmax(-1).squeeze().tolist()
# for span, predicted_class_idx in zip(entity_spans, predicted_class_indices):
#   if predicted_class_idx != 0:
#     print(text[span[0]:span[1]], model.config.id2label[predicted_class_idx])
